In [1]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import numpy as np
dataset_labeled = pd.read_csv('HotelReservations.csv').drop(['Booking_ID'], axis=1)
dataset_labeled

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled,label_avg_price_per_room
0,2,0,1,2,0,224,2017,10,2,0,...,False,False,False,False,False,True,False,False,True,1
1,2,0,2,3,0,5,2018,11,6,0,...,False,False,False,False,False,False,True,False,True,2
2,1,0,2,1,0,1,2018,2,28,0,...,False,False,False,False,False,False,True,True,False,1
3,2,0,0,2,0,211,2018,5,20,0,...,False,False,False,False,False,False,True,True,False,2
4,2,0,1,1,0,48,2018,4,11,0,...,False,False,False,False,False,False,True,True,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,0,85,2018,8,3,0,...,False,False,False,False,False,False,True,False,True,3
36271,2,0,1,3,0,228,2018,10,17,0,...,False,False,False,False,False,False,True,True,False,2
36272,2,0,2,6,0,148,2018,7,1,0,...,False,False,False,False,False,False,True,False,True,2
36273,2,0,0,3,0,63,2018,4,21,0,...,False,False,False,False,False,False,True,True,False,2


Reposiciona label a ser classificada na primeira posição

In [3]:
dataset_labeled.columns
columns = []

columns.append('label_avg_price_per_room')
for i in range(len(dataset_labeled.columns[:-1])):
    columns.append(dataset_labeled.columns[i])

dataset_labeled = dataset_labeled[columns]
dataset_labeled

,label_avg_price_per_room,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,...,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled
0,1,2,0,1,2,0,224,2017,10,2,...,False,False,False,False,False,False,True,False,False,True
1,2,2,0,2,3,0,5,2018,11,6,...,False,False,False,False,False,False,False,True,False,True
2,1,1,0,2,1,0,1,2018,2,28,...,False,False,False,False,False,False,False,True,True,False
3,2,2,0,0,2,0,211,2018,5,20,...,False,False,False,False,False,False,False,True,True,False
4,2,2,0,1,1,0,48,2018,4,11,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,3,0,2,6,0,85,2018,8,3,...,False,False,False,False,False,False,False,True,False,True
36271,2,2,0,1,3,0,228,2018,10,17,...,False,False,False,False,False,False,False,True,True,False
36272,2,2,0,2,6,0,148,2018,7,1,...,False,False,False,False,False,False,False,True,False,True
36273,2,2,0,0,3,0,63,2018,4,21,...,False,False,False,False,False,False,False,True,True,False


In [4]:
# let variables between 0-2 for xgboost use
def xgboost_label(room_price_label):
    if room_price_label == 1:
        return 0
    elif room_price_label == 2:
        return 1
    else:
        return 2

In [5]:
dataset_labeled['label_avg_price_per_room'] = dataset_labeled['label_avg_price_per_room'].apply(lambda x: xgboost_label(x))

In [6]:
dataset_labeled

,label_avg_price_per_room,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,...,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled
0,0,2,0,1,2,0,224,2017,10,2,...,False,False,False,False,False,False,True,False,False,True
1,1,2,0,2,3,0,5,2018,11,6,...,False,False,False,False,False,False,False,True,False,True
2,0,1,0,2,1,0,1,2018,2,28,...,False,False,False,False,False,False,False,True,True,False
3,1,2,0,0,2,0,211,2018,5,20,...,False,False,False,False,False,False,False,True,True,False
4,1,2,0,1,1,0,48,2018,4,11,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,2,3,0,2,6,0,85,2018,8,3,...,False,False,False,False,False,False,False,True,False,True
36271,1,2,0,1,3,0,228,2018,10,17,...,False,False,False,False,False,False,False,True,True,False
36272,1,2,0,2,6,0,148,2018,7,1,...,False,False,False,False,False,False,False,True,False,True
36273,1,2,0,0,3,0,63,2018,4,21,...,False,False,False,False,False,False,False,True,True,False


In [7]:
from sklearn.model_selection import train_test_split

df_treinamento, df_teste = train_test_split(dataset_labeled, test_size=0.3, random_state=40)

In [8]:
df_treinamento.shape

(25392, 32)

In [9]:
df_teste.shape

(10883, 32)

Transforma no array do numpy com os atributos previsores.

In [10]:
X_teste = df_teste.iloc[:,1:len(dataset_labeled)].astype('float32').values
X_teste

array([[2., 0., 2., ..., 0., 0., 1.],
       [2., 1., 2., ..., 1., 1., 0.],
       [2., 0., 2., ..., 1., 1., 0.],
       ...,
       [2., 0., 2., ..., 1., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [11]:
X_teste.shape

(10883, 31)

Transforma no array do numpy com os atributos classificatório.

In [12]:
Y_teste = df_teste.iloc[:, 0].astype('float32').values
Y_teste

array([0., 2., 2., ..., 1., 0., 1.], dtype=float32)

In [13]:
Y_teste.shape

(10883,)

In [14]:
df_treinamento.astype('float32').to_csv('hotel-reservations-train-xgboost.csv', header = False, index = False)
df_teste.astype('float32').to_csv('hotel-reservations-test-xgboost.csv', header = False, index = False)

In [15]:
import collections

collections.Counter(df_treinamento['label_avg_price_per_room'])

Counter({1: 9473, 2: 8276, 0: 7643})

In [16]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

smote = SMOTE()

In [17]:
X_resampled, y_X_resampled = smote.fit_resample(df_treinamento.iloc[:,1:len(dataset_labeled)].astype('float32').values, df_treinamento.iloc[:, 0].astype('float32').values)

In [18]:
collections.Counter(y_X_resampled)

Counter({1.0: 9473, 0.0: 9473, 2.0: 9473})

In [19]:
t = pd.DataFrame(X_resampled)
t.insert(0, column=1000, value = y_X_resampled)
t

,1000,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,1.0,2.000000,0.0,0.000000,3.000000,0.0,37.000000,2018.000000,10.000000,13.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.000000
1,0.0,1.000000,0.0,0.000000,1.000000,0.0,2.000000,2018.000000,4.000000,20.000000,...,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,1.000000
2,1.0,2.000000,0.0,0.000000,2.000000,0.0,137.000000,2018.000000,11.000000,15.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.000000
3,1.0,1.000000,0.0,2.000000,1.000000,0.0,32.000000,2018.000000,4.000000,17.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.000000
4,0.0,2.000000,0.0,2.000000,5.000000,0.0,142.000000,2018.000000,8.000000,22.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28414,2.0,2.000000,0.0,0.233408,1.533184,0.0,38.000000,2018.000000,8.766592,15.766592,...,0.766592,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.233408,0.766592
28415,2.0,1.618634,2.0,0.762733,3.618634,0.0,9.618634,2017.381348,8.000000,25.000000,...,0.000000,0.618634,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,1.000000
28416,2.0,3.000000,0.0,1.085400,4.085400,0.0,194.829193,2018.000000,8.914600,5.170800,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.000000
28417,2.0,2.000000,0.0,0.000000,3.000000,0.0,96.000000,2018.000000,9.734212,13.000000,...,0.265788,0.734212,0.0,0.0,0.0,0.0,0.0,1.0,0.265788,0.734212


In [20]:
t.to_csv('hotel-reservations-train-xgboost.csv', header = False, index = False)

### Configurações SageMaker


In [21]:
import sagemaker
import boto3
from sagemaker import Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [22]:
# set default credentials
boto3.setup_default_session(region_name='us-east-1')

# create session
session = sagemaker.Session()

# define bucket, folders and keys
bucket = 'sprint0405'
subpasta_modelo = 'modelos/hotelreservations/xgboost-oversampling'
subpasta_dataset = 'dataset/hotelreservations'
key_train = 'hotel-train-data-xgboost'
key_test = 'hotel-test-data-xgboost'

role = sagemaker.get_execution_role()

# paths for train and test
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)

print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

Role: arn:aws:iam::891377075477:role/service-role/AmazonSageMaker-ExecutionRole-20240517T101645
Localização da base de treinamento: s3://sprint0405/dataset/hotelreservations/train/hotel-train-data-xgboost
Localização da base de teste: s3://sprint0405/dataset/hotelreservations/test/hotel-test-data-xgboost
Modelo final será salvo em: s3://sprint0405/modelos/hotelreservations/xgboost-oversampling/output


In [23]:
import os
with open('hotel-reservations-train-xgboost.csv', 'rb') as f:
  boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [24]:
with open('hotel-reservations-test-xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)

### Treinamento do XGBoost

In [25]:
from sagemaker import image_uris
container = image_uris.retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='latest')

In [26]:
xgboost = sagemaker.estimator.Estimator(
    image_uri = container,
    role = role,
    instance_count = 1,
    instance_type = 'ml.m5.2xlarge',
    output_path = output_location,
    sagemaker_session = session
)

In [27]:
xgboost.set_hyperparameters(
    num_round=200,
    objective='multi:softprob',
    num_class=3
)

In [28]:
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type = 'S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type = 'S3Prefix')
data_channels = {'train': train_input}

In [29]:
xgboost.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-2024-05-23-17-24-27-543


2024-05-23 17:24:27 Starting - Starting the training job...
2024-05-23 17:24:41 Starting - Preparing the instances for training...
2024-05-23 17:25:28 Downloading - Downloading the training image......
2024-05-23 17:26:03 Training - Training image download completed. Training in progress.Arguments: train
[2024-05-23:17:26:22:INFO] Running standalone xgboost training.
[2024-05-23:17:26:22:INFO] Path /opt/ml/input/data/validation does not exist!
[2024-05-23:17:26:22:INFO] File size need to be processed in the node: 5.61mb. Available memory size in the node: 23880.84mb
[2024-05-23:17:26:22:INFO] Determined delimiter of CSV input is ','
[17:26:22] S3DistributionType set as FullyReplicated
[17:26:22] 56838x31 matrix with 1307274 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[17:26:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[17:26:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 11

### Deploy e avaliação

In [30]:
xgboost_regressor = xgboost.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-05-23-17-27-53-506
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-05-23-17-27-53-506
INFO:sagemaker:Creating endpoint with name xgboost-2024-05-23-17-27-53-506


------!

In [31]:
from sagemaker.serializers import CSVSerializer
xgboost_regressor.serializer = CSVSerializer()

In [32]:
X_teste.shape, type(X_teste)

((10883, 31), numpy.ndarray)

In [33]:
previsoes_bytes = xgboost_regressor.predict(X_teste)
previsoes_str = previsoes_bytes.decode('utf-8')
previsoes_list = eval(previsoes_str)
previsoes_array = np.array(previsoes_list)
previsoes_array

array([[7.76154339e-01, 2.12254599e-01, 1.15910545e-02],
       [3.29464383e-05, 3.28809326e-03, 9.96678948e-01],
       [5.64677492e-02, 3.91164392e-01, 5.52367806e-01],
       ...,
       [2.03458816e-01, 6.71926677e-01, 1.24614477e-01],
       [7.46401250e-01, 2.51241654e-01, 2.35715089e-03],
       [5.33229291e-01, 4.62213129e-01, 4.55757510e-03]])

In [34]:
predicted_labels = np.argmax(previsoes_array, axis=1)
predicted_labels

array([0, 2, 2, ..., 1, 0, 0])

In [35]:
Y_teste = np.array(Y_teste)
Y_teste

array([0., 2., 2., ..., 1., 0., 1.], dtype=float32)

In [36]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

accuracy = accuracy_score(Y_teste, predicted_labels)
precision = precision_score(Y_teste, predicted_labels, average='macro')
recall = recall_score(Y_teste, predicted_labels, average='macro')
f1 = f1_score(Y_teste, predicted_labels, average='macro') 

In [37]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.8412202517688138
Precision: 0.8444696474294306
Recall: 0.843794748232629
F1 Score: 0.8436825459179724


In [38]:
print(classification_report(Y_teste, predicted_labels))

              precision    recall  f1-score   support

         0.0       0.88      0.83      0.86      3308
         1.0       0.81      0.80      0.80      4127
         2.0       0.85      0.90      0.87      3448

    accuracy                           0.84     10883
   macro avg       0.84      0.84      0.84     10883
weighted avg       0.84      0.84      0.84     10883



In [39]:
cm = confusion_matrix(Y_teste, predicted_labels)
cm

array([[2760,  475,   73],
       [ 335, 3304,  488],
       [  40,  317, 3091]])